# sfapi_client Demo

We'll start off by importing some libraries we'll be using in the demo

In [ ]:
from sfapi_client import Client
from sfapi_client.compute import Machine
from sfapi_client.jobs import JobCommand, JobState

from datetime import datetime
from pathlib import Path
from authlib.jose import JsonWebKey
import json
from io import BytesIO

***
# Exercise 1 - Un-Authenticated Client
## Check NERSC Status
### These can all be done without a superfacility client
***
Before we start any computing, let's check that Perlmutter is up.

In [ ]:
with Client() as client:
    perlmutter_status = client.resources.status(Machine.perlmutter)

perlmutter_status.status

We can also check the status of other systems under `resources` in the client

In [ ]:
with Client() as client:
    nersc_status = client.resources.status()

# For each of the resources print the status
for name, status in nersc_status.items():
    print(f"{name: <22}| {status.description: <25}| {status.status}")

This can also be used to get past and upcoming `outages` 

In [ ]:
with Client() as client:
    # Similiar to before but this time let's get the outages
    nersc_status = client.resources.outages()

# The `nersc_status` object is a dictionary of lists of outages
# Get the list you want from the dictionary based on the name
print(nersc_status["perlmutter"])

In [ ]:
# We can check any past or upcoming outages this month
today = datetime.now()
for outage in nersc_status["perlmutter"]:
    if today.month == outage.start_at.month and today.year == outage.start_at.year:
        print(outage)


What if we try to get more information from the API with our client?

In [ ]:
with Client() as client:
    user = client.user()
    # We get an error!


# 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Exercise 2 - Authenticated Client 
## Setup keys and get user and project information
***
Let's setup keys so we can get some more information
I've stored my key is stored in a file in `~/.superfacility/`. Change the path below to where you stored your keys

In [ ]:
sf_key_dir = Path().home() / ".superfacility"

for sf_file in sf_key_dir.iterdir():
    if sf_file.is_file():
        sf_file.chmod(0o600)
        print(sf_file.name)

In [ ]:
# Paste the client_id for the key here
client_id = (sf_key_dir / "clientid.txt").read_text()

# Get the path for your json file here
sfapi_key = sf_key_dir / "priv_key.jwk"

# This opens the json file and reads it into a format the client understands
client_secret = JsonWebKey.import_key(json.loads(sfapi_key.read_text()))

### We'll use these `client_id` and `client_secret` through the rest of our tutorial
Lets make sure we're authenticated and check some information from the API

In [ ]:
# Create a client
with Client(client_id, client_secret) as client:
    # Get the user info, "Who does the api think I am?"
    user = client.user()

### All data returned from the API is in an object 
Obejects are specific for the type of information returned from the API and can be used to get it's attributes or can be returned as dictionaries or json.

In [ ]:
# We got a user object
print(type(user).__name__)

# The user object has an attribute name which is the nersc username
print(user.name)

# Or we can return json which can be helpful for serializing to other libraries
print(user.model_dump_json(exclude=["client", "workphone", "otherPhones"]))


### We can also get other information from the client about `projects` and `groups` and API `clients` that are associated with the authenticated `user`
This is all information you can get from iris.nersc.gov, exposed programatically through the API. 

In [ ]:
with Client(client_id, client_secret) as client:
    # First get the user object
    user = client.user()
    # Get projects associated with user, hours and CFS storage
    projects = user.projects()
    # Get groups assocaited with the user
    groups = user.groups()
    
    clients = user.clients()

# This is the client I made for the sfapi_training!
clients[0]


In [ ]:
# For each project
# Print the information you want
print("Project name |        Hours Given | Hours Remaining")
for project in projects:
    print("=" * 51)
    print(
        f"{project.repo_name: <13}| {project.hours_given:>12.2f} Hours | {project.hours_given - project.hours_used:>8.2f} Hours"
    )


# 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Exercise 3 - Filesystem interactions and small file upload/download
## Interact with NERSC Data Transfer Nodes 
***
Now that we have an authneticated client we can interact with NERSC systems

Let's make some useful variables for our home and scratch directory that we'll use in the next exercises

Your home and scratch paths are based on your username 

* `/global/homes/username_first_letter/username`
* `/pscratch/sd/username_first_letter/username`

Bonus points for using the `user` object to automatically generate it

In [ ]:
home = f"/global/homes/{user.name[0]}/{user.name}"
scratch = f"/pscratch/sd/{user.name[0]}/{user.name}"

Lets make a directory in `$SCRATCH` to put things from the API training demos. We'll use the Data Transfer Nodes for doing these filesystem operations.

In [ ]:
with Client(client_id, client_secret) as client:
    dtns = client.compute(Machine.dtns)
    # This will run a command on dtns, here we use `mkdir` to make our output directory
    dtns.run(f"mkdir -p {scratch}/sfapi-demo")
    # We can run ls on the directory to see that it was created
    [output_dir] = dtns.ls(f"{scratch}/sfapi-demo", directory=True)

# Check that the directory is there
output_dir.is_dir()

Next lets upload a small file to that directory and make sure it's there. 

Since we're uploading a new file that's not there we'll need to get the directory we want to upload the file to with `directory=True` and make sure the file has a `filename` assocaited with it.

In [ ]:
# What we want in our file
file_contents = "hello world!"

my_input_file = BytesIO(file_contents.encode())
# Give our BytesIO file a filename to upload to
my_input_file.filename = "hello.txt"

with Client(client_id, client_secret) as client:
    dtns = client.compute(Machine.dtns)
    print(f"There are {len(dtns.ls(f'{scratch}/sfapi-demo'))} files in the directory")
    [input_file_dir] = dtns.ls(f"{scratch}/sfapi-demo", directory=True)
    # Upload the input file to the directoy object
    input_file_dir.upload(my_input_file)
    print(f"Now there's {len(dtns.ls(f'{scratch}/sfapi-demo'))} files in the directory")


Lets verify that we put some text in that file

In [ ]:
with Client(client_id, client_secret) as client:
    dtn = client.compute(Machine.dtns)
    # Search for the file with ls
    [out_file] = dtn.ls(f"{scratch}/sfapi-demo/hello.txt")
    # Then download and read the file
    contents = out_file.download()
    print(contents.read())


Once the file is there we can easily upload to it again, but be careful as the file will be completly overwriten with out new contents.

In [ ]:
new_file_contests = """

Hello from the API!

"""

with Client(client_id, client_secret) as client:
    dtn = client.compute(Machine.dtns)
    # ls the file we want to upload to
    [input_file] = dtn.ls(f"{scratch}/sfapi-demo/hello.txt")
    # Upload the new contents
    input_file.upload(BytesIO(new_file_contests.encode()))

    contents = input_file.download()
    print(contents.read())


# 

# 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Exercise 4 - Interacting with Perlmutter
## Getting job information and submitting batch work
***

Now we'll connect to perlmutter and interact with Slurm to get information about past jobs as well as submit work to Slurm. Lets check how many jobs are currently running.

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    current_jobs = perlmutter.jobs(command=JobCommand.squeue)

running_jobs = [job for job in current_jobs if job.state == JobState.RUNNING]
len(running_jobs)


#

Now lets run a job and see how to interact with the job with the API. We'll start with a very simple python code to generate random numbers.

In [ ]:
N = 10000
account = project.name
qos = "debug"

job_script = f"""#!/bin/bash

#SBATCH -q {qos}
#SBATCH -A {account}
#SBATCH -N 1
#SBATCH -n 1
#SBATCH -C cpu
#SBATCH -t 00:02:00
#SBATCH -J sfapi-demo
#SBATCH --chdir={scratch}/sfapi-demo
#SBATCH --output={scratch}/sfapi-demo/sfapi-demo-%j.out
#SBATCH --error={scratch}/sfapi-demo/sfapi-demo-%j.error

module load python
# Prints N random numbers to form a normal disrobution
python -c "import numpy as np; numbers = np.random.normal(size={N}); [print(n) for n in numbers]"
"""

print(job_script)


In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    # Jobs can be submitted from
    job = perlmutter.submit_job(job_script)
    # Let's save the job id to use later
    job_id = job.jobid
    print(f"Started {job_id}!")


In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    job = perlmutter.job(jobid=job_id)
    print(job.state)

You can also have the client wait for the job to complete. It will poll the API until the job is in a ternimal state.

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    job = perlmutter.job(jobid=job_id)
    job.complete()


Now the job is done let's download the results file and read it into a variable.

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    [output_file] = perlmutter.ls(f"{scratch}/sfapi-demo/sfapi-demo-{job.jobid}.out")
    print(f"Is the file there? {output_file.is_file()}")
    
    output_file_numbers = output_file.download()
    output_numbers = output_file_numbers.read()

Then we can look at our output results from the job we ran.

In [ ]:
import matplotlib.pyplot as plt

# Convert text numbers into a list we can use with matplotlib
numers = list(map(float, output_numbers.split("\n")[:-1]))

plt.hist(numers, bins=100)
plt.show()